In [6]:
import hashlib
import datetime
import time
import json

# 難易度の定義 この場合は、最低６つの0が並ぶことが条件となる。
INITIAL_BITS = 0x1d777777
MAX_32BIT = 0xffffffff

# ブロッククラス
class Block():
    def __init__(self, index, prev_hash, data, timestamp, bits):
        self.index = index
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp
        self.bits = bits
        self.nonce = 0
        self.elapsed_time = ""
        self.block_hash = ""
    
    def __setitem__(self, key, value):
        setattr(self, key, value)
        
    def to_json(self):
        return{
            "index" : self.index,
            "prev_hash": self.prev_hash,
            "stored_data" : self.data,
            "timestamp" : self.timestamp.strftime("%Y/%m/%d %H:%M:%S"),
            "bits" : hex(self.bits)[2:].rjust(8, "0"),
            "nonce" : hex(self.nonce)[2:].rjust(8, "0"),
            "elapsed_time" : self.elapsed_time,
            "block_hash" : self.block_hash
        }
    
    # ブロックチェーンを計算するメソッド
    def calc_blockhash(self):
        # ブロックヘッダー
        blockheader = str(self.index) + str(self.prev_hash) + str(self.data) + str(self.timestamp) + hex(self.bits)[2:] + str(self.nonce)
        h = hashlib.sha256(blockheader.encode()).hexdigest()
        self.block_hash = h
        return h
    
    # 難易度を計算するメソッド
    def calc_target(self):
        exponent_bytes = (self.bits >> 24) - 3
        exponent_bits = exponent_bytes * 8
        coefficient = self.bits & 0xffffff
        return coefficient << exponent_bits
    
    def check_valid_hash(self):
        return int(self.calc_blockhash(), 16) <= self.calc_target()

# ブロックチェーンクラス
class Blockchain():
    def __init__(self,initial_bits):
        self.chain = []
        self.initial_bits = initial_bits
        
    def add_block(self, block):
        self.chain.append(block)
        
    def getblockinfo(self, index=-1):
        return print(json.dumps(self.chain[index].to_json(), indent = 2, sort_keys = True, ensure_ascii = False))
    
    # マイニングするメソッド
    def mining(self, block):
        start_time = int(time.time() * 1000)
        while True:
            for n in range(MAX_32BIT + 1):
                block.nonce = n
                if block.check_valid_hash():
                    end_time = int(time.time() * 1000)
                    block.elapsed_time = str((end_time - start_time) /  1000.0) + "秒"
                    self.add_block(block)
                    self.getblockinfo()
                    return 
                new_time  = datetime.datetime.now()
                
                if new_time == block.timestamp:
                    block.timestamp += datetime.timedelta(seconds=1)
                else:
                    block.timestamp = new_time
    
    # ジェネシスブロックを生成するためのメソッド
    def create_genesis(self):
        genesis_block = Block(0, "0000000000000000000000000000000000000000000000000000000000000000", "ジェネシスブロック", datetime.datetime.now(), self.initial_bits)
        self.mining(genesis_block)
    
    # 新しいブロックを追加するためのメソッド
    def add_newblock(self, i):
        last_block = self.chain[-1]
        # 新しい難易度を定義するためのメソッドを呼び出す。
        new_bits = self.get_retarget_bits()
        # new_bitsには、新しい難易度か-1が格納されているので、分岐する。
        if new_bits < 0:
            bits = last_block.bits
        else :
            bits = new_bits
        
        block = Block(i + 1, last_block.block_hash, "ブロック" + str(i + 1), datetime.datetime.now(), last_block.bits)
        self.mining(block)
        
    #  難易度調整のためのメソッド
    def get_retarget_bits(self):
        # 難易度調整の要不要のジャッジ
        if len(self.chain) == 0 or len(self.chain) % 5 != 0 :
            return -1
        # 5ブロックごとに難易度調整が行われるようにする。
        expected_time  = 140 * 5
        
        # 以下、マイニングの所要時間の計算
        if len(self.chain) != 5:
            first_block = self.chain[-(1 + 5)]
        else:
            first_block = self.chain[0]
        last_block = self.chain[-1]
        
        # first_blockとlast_blockからそれぞれタイムスタンプを取得する。
        first_time = first_block.timestamp.timestamp() 
        last_time = last_block.timestamp.timestamp()
        # 経過時間を計算する、
        total_time = last_time - fisrt_time
        
        # ターゲットの再計算
        target = last_block.calc_target()
        # マイニングにかかった時間を比較している。
        delta = total_time / expected_time
        # 差異の上限値と下限値の調べて、それぞれリミッターを設ける。
        if delta < 0.25:
            delta = 0.25
        if delta > 4:
            delta = 4
        # 新しいターゲットを計算する。
        new_target = int(target * delta)
        # 新しいbitを計算する。
        exponent_bytes = (last_block.bits >> 24) - 3
        exponent_bits =  exponent_bytes * 8
        temp_bits = new_target >> exponent_bits
        # 大きすぎる場合
        if temp_bits != temp_bits & 0xffffff :
            exponent_bytes += 1
            exponent_bits += 8
             # 小さすぎる場合
        elif temp_bits == temp_bits & 0xffff :
            exponent_bytes -= 1
            exponent_bits -= 8
        return ((exponent_bytes + 3) << 24 | (new_target >> exponent_bits)) # ←新しいbitsの計算
        
        
#ブロックチェーンクラスのインスタンス化
if __name__ == "__main__" :
    bc = Blockchain(INITIAL_BITS)
    print("ジェネシスブロックを作成中・・・")
    bc.create_genesis ()
    for i in range(30):
        print(str(i + 2) + "番目のブロックを作成中・・・・")
        bc.add_newblock(i)
        
        
        
        
        
        
        
        

ジェネシスブロックを作成中・・・
{
  "bits": "1d777777",
  "block_hash": "0000000c4586a673f95fa11f1b4d5cdc4a7c818afad405b1209e77202ab71926",
  "elapsed_time": "65.88秒",
  "index": 0,
  "nonce": "00caf231",
  "prev_hash": "0000000000000000000000000000000000000000000000000000000000000000",
  "stored_data": "ジェネシスブロック",
  "timestamp": "2020/12/05 22:27:48"
}
2番目のブロックを作成中・・・・
{
  "bits": "1d777777",
  "block_hash": "0000004cf64a98a2657ffa5a1154caa4cfcd7f0005eb8833d205882c5ca4dcfe",
  "elapsed_time": "113.943秒",
  "index": 1,
  "nonce": "01659cd3",
  "prev_hash": "0000000c4586a673f95fa11f1b4d5cdc4a7c818afad405b1209e77202ab71926",
  "stored_data": "ブロック1",
  "timestamp": "2020/12/05 22:29:42"
}
3番目のブロックを作成中・・・・
{
  "bits": "1d777777",
  "block_hash": "00000049a5baec0bba011344211af7896a36da11cb926c5bc3a1f7139a195213",
  "elapsed_time": "106.263秒",
  "index": 2,
  "nonce": "014ab7ee",
  "prev_hash": "0000004cf64a98a2657ffa5a1154caa4cfcd7f0005eb8833d205882c5ca4dcfe",
  "stored_data": "ブロック2",
  "timestamp": "2

NameError: name 'fisrt_time' is not defined